In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

import glob
from pathlib import Path

import ta

import yfinance as yf
import nasdaqdatalink
import pandas_datareader.famafrench as ff
import pandas_datareader.data as web


In [ ]:
path_big = "data/stockdata_bigcap/*.csv"
path_small = "data/stockdata_smallcap/*.csv"

stockdata_feature_small_dict = {}
stockdata_feature_big_dict = {}

for filename in glob.glob(path_small):
    symbol = Path(filename).stem
    single_df = pd.read_csv("data/stockdata_feature_full/" + symbol +".csv", index_col="Date")
    single_df.index = pd.to_datetime(single_df.index)
    stockdata_feature_small_dict[symbol] = single_df

for filename in glob.glob(path_big):
    symbol = Path(filename).stem
    single_df = pd.read_csv("data/stockdata_feature_full/" + symbol +".csv", index_col="Date")
    single_df.index = pd.to_datetime(single_df.index)
    stockdata_feature_big_dict[symbol] = single_df

In [ ]:
panel_bigcap = pd.concat([frame for frame in stockdata_feature_big_dict.values()])
panel_smallcap = pd.concat([frame for frame in stockdata_feature_small_dict.values()])


In [ ]:
micro_cols = [
    "PPO","PVO","MOM10d","MOM60d","RSI","SRSI","SO","TSI","UO","WR","EMV","FI","MFI","NVI","BB","BBW",
    "DC","KC","UI","AI","CCI","KST","MACD","MI","STC","TRIX","VI","BETA60d","BETA20d","BETA20d^2",
    "CHBETA","VOLA20d","VOLA60d","CHVOLA","MAXRET","MINRET","CHMOM","RETURN"
]

macro_cols = [
    "MRKTMOM10d","MRKTMOM40d","MRKTVOLA","MRKT","DIVYIELD","EARNYIELD","PERATIO","T10Y2Y","TB6TB3",
    "TB3","TB6","PRIMETB3","FFTB3","SENT1d","SENTCUM","SENTMA","CPI","UNRATE","RF"
]

In [ ]:
for frame in (panel_bigcap, panel_smallcap):
    for col_macro in macro_cols:
        for col_micro in micro_cols:
            frame[col_micro +"x"+ col_macro] = frame[col_macro]*frame[col_micro]
    frame.drop(macro_cols, axis=1, inplace=True)

In [ ]:
for frame in (panel_bigcap, panel_smallcap):
    for date in frame.index.unique():
        for feature in frame.drop("EXCESS_RETURN_T+1").columns:
            crosssec_min = frame.loc[date, feature].values.min()
            crosssec_max = frame.loc[date, feature].values.max()
            frame.loc[date, feature] = frame.loc[date, feature].apply(
                lambda x: 2 * (x - crosssec_min) / (crosssec_max - crosssec_min) - 1
            )

In [ ]:
panel_smallcap.round(5)
panel_smallcap.round(5)
panel_full = pd.concat(panel_bigcap, panel_smallcap)